# Projeto Final - Machine Learning:
#### Integrantes: 
    - Natália Queiroz Menezes Carreras
    - Willian Kenzo Asanuma Lee
#### Database: https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset

#### Bibliotecas necessárias:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt